# Informatica PowerCenter to Databricks Transformation

This notebook reads the PowerCenter XML mapping definition and creates equivalent transformation logic in PySpark.

## Overview

The original mapping (`mp_stg_TRANSACTION_CAMS_DD`) extracts data from TBHRD_TRANSACTION_CAMS_DD and loads it to TBST2_TRANSACTION_CAMS_DD with multiple column transformations. Key operations include:

1. Reading from Oracle source table
2. Data type conversions
3. String trimming
4. Date format conversions
5. NULL handling
6. Data masking for sensitive data
7. Writing to target table

## 1. Setup and Configuration

First, we'll import necessary libraries and define configuration parameters

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import hashlib
from datetime import datetime
import xml.etree.ElementTree as ET
import os

# Configuration parameters
source_table = "TBHRD_TRANSACTION_CAMS_DD"
target_table = "TBST2_TRANSACTION_CAMS_DD"
source_database = "ODS_Sviluppo"
source_schema = "HRD_CAMS_OBJ"

# Date parameters (will be passed as parameters in ADF)
p_dt_inf = spark.conf.get("spark.databricks.job.p_dt_inf", "2025-03-01 00:00:00")
p_dt_sup = spark.conf.get("spark.databricks.job.p_dt_sup", "2025-03-21 23:59:59")

## 2. JDBC Connection Setup

Configure JDBC connection for Oracle source database. In a production environment, secrets would be retrieved from Azure Key Vault.

In [ ]:
# JDBC connection configuration
jdbc_url = f"jdbc:oracle:thin:@//your-oracle-server:1521/{source_database}"
connection_properties = {
    "user": "dbutils.secrets.get(scope='oracle-keys', key='username')",
    "password": "dbutils.secrets.get(scope='oracle-keys', key='password')",
    "driver": "oracle.jdbc.driver.OracleDriver",
    "fetchsize": "10000"
}

## 3. Source Query Definition

Define the source query based on the Informatica mapping's Source Qualifier

In [ ]:
# Build query similar to the SQL in Source Qualifier
query = f"""
SELECT 
  {source_table}.DATA_APP,
  {source_table}.COMPANY_GROUPS,
  {source_table}.AMBIENTE,
  {source_table}.POOL,
  {source_table}.COMPANY,
  {source_table}.AC_GRP_INTRL_ID,
  {source_table}.AC_INTRL_ID,
  {source_table}.TX_CYC_BGN_DT,
  {source_table}.TX_GU_EFF_DT,
  {source_table}.JO_PSTG_SEQ_NR,
  {source_table}.JO_INTRL_SEQ_NR,
  {source_table}.JO_PSTG_CD,
  {source_table}.JO_PSTG_ID,
  standard_hash(TRIM({source_table}.JO_PSTG_ID), 'MD5') AS ID_CAR,
  standard_hash(TRIM({source_table}.JO_PSTG_ID), 'SHA512') AS CO_HASHED_PAN,
  SUBSTR(TRIM({source_table}.JO_PSTG_ID), 0, 6) || LPAD(SUBSTR(TRIM({source_table}.JO_PSTG_ID), LENGTH(TRIM({source_table}.JO_PSTG_ID))-3, LENGTH(TRIM({source_table}.JO_PSTG_ID))), LENGTH(SUBSTR(TRIM({source_table}.JO_PSTG_ID), 7, LENGTH(TRIM({source_table}.JO_PSTG_ID)))), '*') AS CO_CAR,
  {source_table}.JO_DB_CR_IN,
  {source_table}.GU_PSTG_DT,
  {source_table}.JO_SRCE_CD,
  {source_table}.JO_CATG_CD,
  {source_table}.JO_LVL_3_CD,
  {source_table}.JO_LVL_4_CD,
  {source_table}.JO_LVL_5_CD,
  {source_table}.JO_PSTD_AM,
  {source_table}.JO_AVAIL_RV_AM,
  {source_table}.JO_PRT_ON_SN_IN
  -- ... remaining fields would be included here
FROM {source_schema}.{source_table}
WHERE TO_TIMESTAMP(TRIM(TC_UPDT_TS), 'YYYY-MM-DD-HH24.MI.SS.FF') >= TO_DATE('{p_dt_inf}', 'YYYY-MM-DD HH24:MI:SS') 
  AND TO_TIMESTAMP(TRIM(TC_UPDT_TS), 'YYYY-MM-DD-HH24.MI.SS.FF') <= TO_DATE('{p_dt_sup}', 'YYYY-MM-DD HH24:MI:SS')
  AND (TRIM(TC_UPDT_TS) = TRIM(TC_CRT_TS))
"""

print("Source query defined")

## 4. Custom Functions for Transformations

Define custom utility functions to simulate Informatica's transformations

In [ ]:
# Register UDFs for transformations similar to Informatica's Expression transformation

# Function to handle date conversion with format validation
@F.udf(returnType=TimestampType())
def safe_to_date(date_str, format_str):
    """Convert string to date with null handling"""
    if date_str is None or date_str.strip() == "":
        return None
    try:
        return datetime.strptime(date_str.strip(), format_str)
    except:
        return None

# Function to replicate standard_hash in Oracle for MD5
@F.udf(returnType=StringType())
def md5_hash(text):
    """Generate MD5 hash similar to Oracle's standard_hash"""
    if text is None:
        return None
    return hashlib.md5(text.encode()).hexdigest()

# Function to replicate standard_hash in Oracle for SHA512
@F.udf(returnType=StringType())
def sha512_hash(text):
    """Generate SHA512 hash similar to Oracle's standard_hash"""
    if text is None:
        return None
    return hashlib.sha512(text.encode()).hexdigest()

# Function to mask PAN numbers (keep first 6 and last 4 digits)
@F.udf(returnType=StringType())
def mask_pan(pan):
    """Mask PAN number by keeping first 6 and last 4 digits"""
    if pan is None or len(pan.strip()) == 0:
        return None
    pan = pan.strip()
    if len(pan) <= 10:  # Not enough to mask meaningfully
        return pan
    prefix = pan[:6]
    suffix = pan[-4:]
    mask_len = len(pan) - 10
    return prefix + '*' * mask_len + suffix

## 5. Data Extraction and Transformation

Read and transform the data from source to target format

In [ ]:
# Read data using the query
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", f"({query})") \
    .option("user", connection_properties["user"]) \
    .option("password", connection_properties["password"]) \
    .option("driver", connection_properties["driver"]) \
    .option("fetchsize", connection_properties["fetchsize"]) \
    .load()

print(f"Retrieved {df.count()} rows from source")

## 6. Apply Transformations

Apply transformations equivalent to the Informatica Expression transformation

In [ ]:
# Apply transformations to match Informatica expressions
transformed_df = df.withColumn("DATA_APP", F.trim(F.col("DATA_APP"))) \
    .withColumn("COMPANY_GROUPS", F.trim(F.col("COMPANY_GROUPS"))) \
    .withColumn("AMBIENTE", F.trim(F.col("AMBIENTE"))) \
    .withColumn("POOL", F.trim(F.col("POOL"))) \
    .withColumn("COMPANY", F.trim(F.col("COMPANY"))) \
    .withColumn("AC_GRP_INTRL_ID", F.col("AC_GRP_INTRL_ID").cast("decimal(13,0)")) \
    .withColumn("AC_INTRL_ID", F.col("AC_INTRL_ID").cast("decimal(13,0)")) \
    .withColumn("TX_CYC_BGN_DT", safe_to_date(F.col("TX_CYC_BGN_DT"), "%Y-%m-%d")) \
    .withColumn("TX_GU_EFF_DT", safe_to_date(F.col("TX_GU_EFF_DT"), "%Y-%m-%d")) \
    .withColumn("JO_PSTG_SEQ_NR", F.col("JO_PSTG_SEQ_NR").cast("decimal(5,0)")) \
    .withColumn("JO_INTRL_SEQ_NR", F.col("JO_INTRL_SEQ_NR").cast("decimal(5,0)")) \
    .withColumn("JO_PSTG_CD", F.trim(F.col("JO_PSTG_CD"))) \
    .withColumn("JO_DB_CR_IN", F.trim(F.col("JO_DB_CR_IN"))) \
    .withColumn("GU_PSTG_DT", safe_to_date(F.col("GU_PSTG_DT"), "%Y-%m-%d")) \
    .withColumn("JO_SRCE_CD", F.trim(F.col("JO_SRCE_CD"))) \
    .withColumn("JO_CATG_CD", F.trim(F.col("JO_CATG_CD"))) \
    .withColumn("JO_LVL_3_CD", F.trim(F.col("JO_LVL_3_CD"))) \
    .withColumn("JO_LVL_4_CD", F.trim(F.col("JO_LVL_4_CD"))) \
    .withColumn("JO_LVL_5_CD", F.trim(F.col("JO_LVL_5_CD"))) \
    .withColumn("JO_PSTD_AM", F.col("JO_PSTD_AM").cast("decimal(15,2)")) \
    .withColumn("JO_AVAIL_RV_AM", F.col("JO_AVAIL_RV_AM").cast("decimal(15,2)")) \
    .withColumn("JO_PRT_ON_SN_IN", F.when(F.col("JO_PRT_ON_SN_IN").isNull() | (F.trim(F.col("JO_PRT_ON_SN_IN")) == ""), F.lit("N")).otherwise(F.trim(F.col("JO_PRT_ON_SN_IN"))))
    # ... additional transformations would be applied for remaining fields

# Rename columns to match target schema if needed
transformed_df = transformed_df.withColumnRenamed("AMBIENTE", "ENVIROMENT")
transformed_df = transformed_df.withColumnRenamed("COMPANY", "AC_CO_NR")
transformed_df = transformed_df.withColumnRenamed("TX_CYC_BGN_DT", "SN_CYC_BGN_DT")
transformed_df = transformed_df.withColumnRenamed("TX_GU_EFF_DT", "GU_EFF_DT")

print("Transformations applied successfully")

## 7. Write Data to Target

Write the transformed data to the target table

In [ ]:
# Write to target table
target_jdbc_url = f"jdbc:oracle:thin:@//your-oracle-server:1521/{source_database}"

transformed_df.write \
    .format("jdbc") \
    .option("url", target_jdbc_url) \
    .option("dbtable", target_table) \
    .option("user", connection_properties["user"]) \
    .option("password", connection_properties["password"]) \
    .option("driver", connection_properties["driver"]) \
    .mode("append") \
    .save()

print(f"Data successfully written to {target_table}")

## 8. XML Parsing Utility

This utility function can parse the Informatica XML directly to generate transformation logic

In [ ]:
def parse_informatica_xml(xml_path):
    """Parse Informatica PowerCenter XML to extract mapping information"""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Find the mapping
    mapping = root.find(".//MAPPING")
    mapping_name = mapping.get("NAME")
    print(f"Found mapping: {mapping_name}")
    
    # Extract source and target information
    source = root.find(".//SOURCE")
    source_name = source.get("NAME")
    source_db = source.get("DBDNAME")
    source_owner = source.get("OWNERNAME")
    
    target = root.find(".//TARGET")
    target_name = target.get("NAME")
    
    # Get source fields
    source_fields = []
    for field in source.findall("./SOURCEFIELD"):
        source_fields.append({
            "name": field.get("NAME"),
            "datatype": field.get("DATATYPE"),
            "precision": field.get("PRECISION"),
            "scale": field.get("SCALE"),
            "keytype": field.get("KEYTYPE")
        })
    
    # Get target fields
    target_fields = []
    for field in target.findall("./TARGETFIELD"):
        target_fields.append({
            "name": field.get("NAME"),
            "datatype": field.get("DATATYPE"),
            "precision": field.get("PRECISION"),
            "scale": field.get("SCALE"),
            "keytype": field.get("KEYTYPE")
        })
    
    # Find transformation expressions
    expr_trans = root.find(".//TRANSFORMATION[@TYPE='Expression']")
    expressions = []
    
    if expr_trans is not None:
        for field in expr_trans.findall("./TRANSFORMFIELD[@PORTTYPE='OUTPUT']"):
            expr = field.get("EXPRESSION")
            if expr:
                expressions.append({
                    "name": field.get("NAME"),
                    "expression": expr,
                    "datatype": field.get("DATATYPE")
                })
    
    return {
        "mapping_name": mapping_name,
        "source": {
            "name": source_name,
            "database": source_db,
            "owner": source_owner,
            "fields": source_fields
        },
        "target": {
            "name": target_name,
            "fields": target_fields
        },
        "expressions": expressions
    }

# Example usage (commented out as XML file not available in notebook context)
# mapping_info = parse_informatica_xml("prova_script_IPWC.XML")
# print(f"Parsed {len(mapping_info['source']['fields'])} source fields")
# print(f"Parsed {len(mapping_info['target']['fields'])} target fields")
# print(f"Parsed {len(mapping_info['expressions'])} expressions")

## 9. Conclusion

This notebook demonstrates how to implement Informatica PowerCenter mapping logic in Databricks using PySpark.

Key benefits of this approach:

1. **Scalability**: PySpark can handle larger volumes of data than traditional ETL tools
2. **Cost Efficiency**: Uses cloud-native processing rather than dedicated ETL servers
3. **Flexibility**: Can be integrated with modern data platforms and processing frameworks
4. **Maintainability**: Code-based approach allows for version control and CI/CD integration
5. **Extensibility**: Easy to add additional transformations and business logic